In [1]:
from pyspark import SparkConf, SparkContext
import collections

conf = SparkConf().setMaster("local").setAppName("RatingsHistogram")
sc = SparkContext(conf = conf)

lines = sc.textFile("../sources/ml-100k/u.data")
ratings = lines.map(lambda x: x.split()[2])
result = ratings.countByValue()

sortedResults = collections.OrderedDict(sorted(result.items()))
for key, value in sortedResults.items():
    print("%s %i" % (key, value))

your 131072x1 screen size is bogus. expect trouble
24/12/14 20:11:45 WARN Utils: Your hostname, MSI resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/14 20:11:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/14 20:11:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


1 6110
2 11370
3 27145
4 34174
5 21201


This code uses the file fakefriends.csv to calculate the average friends by age.

In [ ]:
from pyspark import SparkConf, SparkContext
import collections
import pandas as pd

if sc:
    sc.stop()
conf = SparkConf().setMaster("local").setAppName("RatingsHistogram")
sc = SparkContext(conf = conf)

def parseLine(line):
    fields = line.split(',')
    age = int(fields[2])
    numFriends = int(fields[3])
    return (age,numFriends)

lines = sc.textFile("../resources/sources/fakefriends.csv")
rdd = lines.map(parseLine)
totalsByAge = rdd.mapValues(lambda x: (x, 1)).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
averagesByAge = totalsByAge.mapValues(lambda x: x[0] / x[1])
results = averagesByAge.collect()
for result in results:
    print(result)


(33, 325.3333333333333)
(26, 242.05882352941177)
(55, 295.53846153846155)
(40, 250.8235294117647)
(68, 269.6)
(59, 220.0)
(37, 249.33333333333334)
(54, 278.0769230769231)
(38, 193.53333333333333)
(27, 228.125)
(53, 222.85714285714286)
(57, 258.8333333333333)
(56, 306.6666666666667)
(43, 230.57142857142858)
(36, 246.6)
(22, 206.42857142857142)
(35, 211.625)
(45, 309.53846153846155)
(60, 202.71428571428572)
(67, 214.625)
(19, 213.27272727272728)
(30, 235.8181818181818)
(51, 302.14285714285717)
(25, 197.45454545454547)
(21, 350.875)
(42, 303.5)
(49, 184.66666666666666)
(48, 281.4)
(50, 254.6)
(39, 169.28571428571428)
(32, 207.9090909090909)
(58, 116.54545454545455)
(64, 281.3333333333333)
(31, 267.25)
(52, 340.6363636363636)
(24, 233.8)
(20, 165.0)
(62, 220.76923076923077)
(41, 268.55555555555554)
(44, 282.1666666666667)
(69, 235.2)
(65, 298.2)
(61, 256.22222222222223)
(28, 209.1)
(66, 276.44444444444446)
(46, 223.69230769230768)
(29, 215.91666666666666)
(18, 343.375)
(47, 233.22222222222

This code takes the temperature for the year 1800, from the csv file 1800.csv, and calculates the minimum temperature for the year 1800

In [26]:
from pyspark import SparkConf, SparkContext
import collections
import pandas as pd

if sc:
    sc.stop()
conf = SparkConf().setMaster("local").setAppName("temperaturesAnalysis")
sc = SparkContext(conf = conf)

def parseLine(line):
    fields = line.split(',')
    stationId = fields[0]
    entryType = fields[2]
    temperature = float(fields[3]) * 0.1 * (9.0 / 5.0) + 32.0
    return (stationId,entryType,temperature)

lines = sc.textFile("../resources/sources/1800.csv")
parsedLines = lines.map(parseLine)
minTemps = parsedLines.filter(lambda x: 'TMIN' in x[1])
stationTemps = minTemps.map(lambda x: (x[0],x[2]))
minYearTemps = stationTemps.reduceByKey(lambda x,y: min(x,y))

results = minYearTemps.collect()
for result in results:
    print(result[0] + "\t{:.2f}F".format(result[1]))

ITE00100554	5.36F
EZE00100082	7.70F


This script finds the number of times a specific word appears on the book inside the sources

In [39]:
import sys
from pyspark import SparkConf, SparkContext
import re

if __name__ == "__main__":
    # Check if the user provided the word as an argument
    if len(sys.argv) != 2:
        print("Usage: python textFind.py <word>")
        sys.exit(1)

    # The word we are searching for
    word_to_find = 'the'#sys.argv[1]


    # Using a regex to remove all non-alphanumeric characters
    def normalize_word(word):
        # Remove all special characters, leaving only letters and digits
        cleaned = re.sub(r'[^A-Za-z0-9]+', '', word)
        return cleaned.lower()
    
    # Stop any existing SparkContext
    if 'sc' in globals():
        sc.stop()

    conf = SparkConf().setMaster("local").setAppName("textFind")
    sc = SparkContext(conf = conf)

    lines = sc.textFile('../resources/sources/Book.txt')
    words = lines.flatMap(lambda x: x.split())
    matchedWords = words.filter(lambda x: normalize_word(x).lower() == word_to_find.lower())
    results = matchedWords.count()
    print(results)
    


1291
